CAS Exam

In [2]:
!pip install openai
!pip install --user -U nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
import os
import re
import openai
from google.colab import drive
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
drive.mount("/content/gdrive/", force_remount=True)

Mounted at /content/gdrive/


In [23]:
%cd /content/gdrive/MyDrive/

/content/gdrive/MyDrive


In [5]:
manual_df = pd.read_csv("cas_exam/cas_manuals.csv")

In [7]:
def remove_urls(text):
    if not text:
        text = ''
    url_pattern = re.compile(r"https?://\S+|www\.\S+")
    return url_pattern.sub(r"", text)

def remove_s(text):
    if not text:
        text = ''
    else:
        _RE_COMBINE_WHITESPACE = re.compile(r"\s+")
        text = _RE_COMBINE_WHITESPACE.sub(" ", text).strip()
    return text

def custom_tokenize(text):
    if not text:
        text = ''
    return word_tokenize(text)

def preprocess(df):
    df["source_content"] = df["source_content"].apply(remove_s)
    df["source_content"] = df["source_content"].apply(remove_urls)
    df['token'] = df["source_content"].apply(custom_tokenize)
    df["token_count"] = df["token"].str.len()
    df = df[(df["token_count"]>= 10)]
    return df

manual_df['source_content'] = manual_df['source_content'].astype(str)
new_manual_df = preprocess(manual_df)
c6_manual_df = new_manual_df.loc[new_manual_df["source"] == "6c_complete_textref.pdf"]
import numpy as np
sub_dfs = np.array_split(c6_manual_df, 10)
index = 0
for sub_df in sub_dfs:
    sub_df.to_csv('cas_exam/cas_manual_6c_'+ str(index) + '.csv', index=False)
    index += 1

In [11]:
index = 0
sub_dfs = []
for i in range(0, 10):
    sub_df = pd.read_csv('cas_exam/cas_manual_6c_'+ str(index) + '.csv')
    sub_dfs.append(sub_df)
    print(sub_df[:3])
    index += 1

   Unnamed: 0                   source  source_page  \
0       16141  6c_complete_textref.pdf            1   
1       16142  6c_complete_textref.pdf            2   
2       16143  6c_complete_textref.pdf            3   

                                      source_content  \
0  SYLLABUS OF BASIC EDUCATION Complete Online Te...   
1  Report on Fundamental Reform of the ALBERTA AU...   
2  The Honourable Travis Toews President of Treas...   

                                               token  token_count  
0  ['SYLLABUS', 'OF', 'BASIC', 'EDUCATION', 'Comp...           14  
1  ['Report', 'on', 'Fundamental', 'Reform', 'of'...           41  
2  ['The', 'Honourable', 'Travis', 'Toews', 'Pres...           45  
   Unnamed: 0                   source  source_page  \
0       17635  6c_complete_textref.pdf          248   
1       17636  6c_complete_textref.pdf          248   
2       17637  6c_complete_textref.pdf          248   

                                      source_content  \
0  f.

In [4]:
#openai.organization = "org-SUudZOFIt405XxJwujsTu5aV"
openai.api_key = ENV

In [36]:
def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

#print(get_embedding("and • Nil for contracts"))
import re
def remove_special_char(text):
    return re.sub('•', ' ', text)

embeded_dfs = []
for i in [9]:
    sub_df = pd.read_csv('cas_exam/cas_manual_6c_'+ str(i) + '.csv')
    #sub_df["source_content"] = sub_df["source_content"].apply(remove_special_char)
    #sub_df['len'] = sub_df["source_content"].str.len()
    #print(i)
    #print(sub_df.loc[sub_df['len']==366]['source_content'].to_list())
    #print(sub_df.loc[sub_df['len']==366]['source_content'])
    print(i, len(sub_df))
    sub_df['ada_embedding'] = sub_df['source_content'].apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
    sub_df.to_csv('cas_exam/embedded_1k_cas_manual_6c_'+ str(i) + '.csv', index=False)
    #embeded_dfs.append(sub_df)
#result = pd.concat(embeded_dfs)
#result.to_csv('cas_exam/embedded_1k_cas_manual_6c.csv', index=False)

9 1420


In [39]:
q_df = pd.read_csv('cas_exam/6c_questions.csv', sep=";")
q_df['q_ada_embedding'] = q_df['question_content'].apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
q_df.to_csv('cas_exam/6c_questions.csv', index=False)

In [40]:
q_df

,exam,semester,question_number,question_content,answer_sample1,answer_sample2,answer_sample3,q_ada_embedding
0,6c,fall_2019,1a,Briefly describe three reasons the insurance i...,Many insurers went bankrupt in 1870’s. Insuran...,NaN,NaN,"[0.004949517548084259, -0.00909569300711155, 0..."
1,6c,fall_2019,1b,Briefly describe a reason guidelines are often...,Changes to legislation must go through the sen...,"Guidelines are subject to interpretation, so m...",Guidelines are interpreted rules on how things...,"[0.003672732273116708, 0.0009299870580434799, ..."
2,6c,fall_2019,1c,Briefly describe four key principles the Offic...,Be forward looking to allow early intervention...,Dynamic adjustments. Sound predictive judgemen...,NaN,"[0.0010426035150885582, -0.011950348503887653,..."


In [41]:
embeded_dfs = []
for i in range(0, 10):
    sub_df = pd.read_csv('cas_exam/embedded_1k_cas_manual_6c_'+ str(i) + '.csv')
    embeded_dfs.append(sub_df)
result = pd.concat(embeded_dfs)
result.to_csv('cas_exam/embedded_1k_cas_manual_6c.csv', index=False)

In [46]:
def str_float(s):
     return [float(x.strip(' []')) for x in s.split(',')]
result['ada_embedding'] = result['ada_embedding'].apply(str_float)
q_df['q_ada_embedding'] = q_df['q_ada_embedding'].apply(lambda x: [float(y) for y in x])

In [50]:
from openai.embeddings_utils import cosine_similarity
def search_docs(doc_df, q_embedding, n=3, pprint=True):
    doc_df['similarities'] = doc_df['ada_embedding'].apply(lambda x: cosine_similarity(x, q_embedding))
    res = doc_df.sort_values('similarities', ascending=False).head(n)[['source', 'source_page', 'source_content', 'similarities']]
    return res

top_dfs = []
test_doc = result[:50]
for index, row in q_df.iterrows():
    print(index)
    top_df = search_docs(result, row['q_ada_embedding'], n=10)
    for col in ['exam', 'semester', 'question_number', 'question_content']:
         top_df[col] = [row[col]] * 10
    top_dfs.append(top_df)
q_doc_df = pd.concat(top_dfs)
q_doc_df.to_csv('cas_exam/6c_questions_w_top10_docs.csv', index=False)

0
1
2


In [51]:
q_doc_df


,source,source_page,source_content,similarities,exam,semester,question_number,question_content
640,6c_complete_textref.pdf,2263,"Over the years, studies have been performed to...",0.847030,6c,fall_2019,1a,Briefly describe three reasons the insurance i...
831,6c_complete_textref.pdf,2299,Solvency II was developed as a group wide solv...,0.839796,6c,fall_2019,1a,Briefly describe three reasons the insurance i...
1348,6c_complete_textref.pdf,1621,"Munch, P., and D. E. Sm allwood, 1982, Theory ...",0.837475,6c,fall_2019,1a,Briefly describe three reasons the insurance i...
400,6c_complete_textref.pdf,1692,State Funds With enactment of state workers co...,0.835267,6c,fall_2019,1a,Briefly describe three reasons the insurance i...
1165,6c_complete_textref.pdf,1605,Regulatory interaction: Insurance is highly re...,0.833626,6c,fall_2019,1a,Briefly describe three reasons the insurance i...
907,6c_complete_textref.pdf,2032,Several high-profile insurance companies engag...,0.833124,6c,fall_2019,1a,Briefly describe three reasons the insurance i...
1202,6c_complete_textref.pdf,214,"Third, the two principal rationales for mainta...",0.832203,6c,fall_2019,1a,Briefly describe three reasons the insurance i...
1401,6c_complete_textref.pdf,1626,The legislation requires that all risk classif...,0.831791,6c,fall_2019,1a,Briefly describe three reasons the insurance i...
994,6c_complete_textref.pdf,1785,"In the recent past, individual insurers have s...",0.830033,6c,fall_2019,1a,Briefly describe three reasons the insurance i...
31,6c_complete_textref.pdf,1858,The legislation is at once very broad and open...,0.829689,6c,fall_2019,1a,Briefly describe three reasons the insurance i...


In [31]:
doc_prompt_cfg = {
        "max_tokens": 3500,
        "text_before_documents": (
            "You are a chatbot assistant answering Casualty Actuarial Society (CAS) exam questions about Regulation and Financial Reporting."
            "You can only respond to a question if the content necessary to answer the question is contained in the following provided documentation. "
            "If the answer is in the documentation, summarize it in a helpful way to the user. "
            "If it isn't, simply reply that you cannot answer the question. "
            "Do not refer to the documentation directly, but use the instructions provided within it to answer questions. "
            "Here is the documentation: "
            "<DOCUMENTS> "
        ),
        "text_before_prompt": (
            "<\DOCUMENTS>\n"
            "REMEMBER:\n"
            "You are a chatbot assistant answering Casualty Actuarial Society (CAS) exam questions about Regulation and Financial Reporting."
            "Here are the rules you must follow:\n"
            "1) You must only respond with information contained in the documentation above. Say you do not know if the information is not provided.\n"
            "2) Do not reference any links, urls or hyperlinks in your answers.\n"
            "3) Do not refer to the documentation directly, but use the instructions provided within it to answer questions.\n"
            "Answer the following question:\n"  
        )
      }

In [27]:
def format_doc(matched_documents, max_tokens):
        """Format our matched documents to plaintext.
        We also make sure they fit in the alloted max_tokens space.
        """
        documents_str = ""
        total_tokens = 0

        num_total_docs = len(matched_documents)
        num_preserved_docs = 0
        for _, row in matched_documents.iterrows():
            doc = row['source_content']
            num_preserved_docs += 1
            token_count = len(doc.split())
            if total_tokens + token_count <= max_tokens:
                documents_str += f"<DOCUMENT>{doc}<\\DOCUMENT>"
                total_tokens += token_count
            else:
                remaining_tokens = max_tokens - total_tokens
                truncated_doc = " ".join(doc.split()[:remaining_tokens])
                documents_str += f"<DOCUMENT>{truncated_doc}<\\DOCUMENT>"
                break

        if num_preserved_docs < (num_total_docs):
            matched_documents = matched_documents.iloc[:num_preserved_docs]

        return documents_str

In [25]:
def format_prompt(documents_str, config):
        """
        Prepare the system prompt with prompt engineering.

        Joins the text before and after documents with
        """
        formatter = "{text_before_documents}\n{documents}\n{text_before_prompt}"
        system_prompt = formatter.format(
            text_before_documents=config["text_before_documents"], documents=documents_str, text_before_prompt=config["text_before_prompt"]
        )
        token_count = len(system_prompt.split())
        if token_count > config["max_tokens"]:
            raise ValueError(f"System prompt tokens > {config['max_tokens']}")
        return system_prompt

In [18]:
simple_prompt_cfg = {
            "max_tokens": 3500,
            "text_before_documents": "You are a chatbot answering questions.\n",
            "text_before_prompt": "Answer the following question:\n",
        }

In [43]:
"""
prompt = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input},
        ]
        return prompt
print(response)
"""
q_doc_df = pd.read_csv('cas_exam/6c_questions_w_top10_docs.csv')
n = 10
max_tokens = 2500
prompt_cfg = doc_prompt_cfg
responses_w_doc = []
responses_wo_doc = []

for q_id in ['1a', '1b', '1c']:
      sub_q_doc_df = q_doc_df.loc[q_doc_df['question_number']==q_id][:n]
      doc_str = format_doc(sub_q_doc_df, max_tokens)
      prompt = format_prompt(doc_str, prompt_cfg)
      prompt_w_doc = prompt + sub_q_doc_df.iloc[0]['question_content']
      prompt_wo_doc = sub_q_doc_df.iloc[0]['question_content']
      #print(len(prompt.split()))
      print("Q:", sub_q_doc_df.iloc[0]['question_content'])
      response = openai.Completion.create(
          model="text-davinci-003",
          prompt=prompt_w_doc,
          temperature=0.5,
          max_tokens=500,
          top_p=1.0,
          frequency_penalty=0.0,
          presence_penalty=0.0
      )
      responses_w_doc.append(response["choices"][0]["text"])
      print("WITH DOC:")
      print(response["choices"][0]["text"])
      response = openai.Completion.create(
          model="text-davinci-003",
          prompt=prompt_wo_doc,
          temperature=0.5,
          max_tokens=500,
          top_p=1.0,
          frequency_penalty=0.0,
          presence_penalty=0.0
      )
      responses_wo_doc.append(response["choices"][0]["text"])
      print("WITHOUT DOC:")
      print(response["choices"][0]["text"])
      print("END")
      print()
q_df = pd.read_csv('cas_exam/6c_questions.csv')
q_df['answer_w_doc_10'] = responses_w_doc
q_df['answer_wo_doc'] = responses_wo_doc

Q: Briefly describe three reasons the insurance industry was singled out for specific solvency legislation in the 1870s.
WITH DOC:


The insurance industry was singled out for specific solvency legislation in the 1870s due to the need for workers compensation insurance, the need for a stable source of affordable insurance coverage, and the fear of insurance companies imposing excessive premium rates that would be a financial burden.
WITHOUT DOC:


1. The insurance industry was growing rapidly and becoming increasingly complex, making it difficult to regulate.

2. Insurance companies were taking on more risk and engaging in more speculative practices, leading to concerns about their solvency.

3. There were numerous cases of insurance companies failing and leaving policyholders without protection, leading to a need for better oversight.
END

Q: Briefly describe a reason guidelines are often favoured over legislation as a form of insurance regulation.
WITH DOC:


Guidelines are often fav

In [44]:
q_df = q_df.drop(columns=['q_ada_embedding'])
q_df.to_csv('cas_exam/6c_questions_with_response.csv', index=False)